In [3]:
# import the necessary libraries
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\ASUSK5~1\miniconda3\envs\medchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
# enter pinecone api key
PINECONE_API_KEY = "ADD YOUR PINECONE API KEY"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [13]:
# Initialize Pinecone client
pc = pinecone.Pinecone()

# list existing indexes
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
# existing_indexes

In [6]:
# declare pinecone index name
mcbot_idx_name = "medical-chatbot"

cloud = os.getenv("PINECONE_CLOUD", "aws")
region = os.getenv("PINECONE_REGION", "us-east-1")

spec = pinecone.ServerlessSpec(cloud=cloud, region=region)

# check whether the index already exists
if mcbot_idx_name not in existing_indexes:
    # create index
    pc.create_index(name=mcbot_idx_name, dimension=1536, metric="cosine", spec=spec)

    while not pc.describe_index(mcbot_idx_name).status["ready"]:
        # wait until index is ready
        time.sleep(1)
    
    print(f"Index {mcbot_idx_name} has been successfully created.")
else:
    print(f"Index {mcbot_idx_name} already exists!")

# connect to the index
index = pc.Index(mcbot_idx_name)

Index medical-chatbot already exists!


In [7]:
# load data from the pdf file
def load_pdf(data_directory):
    loader = DirectoryLoader(data_directory,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [8]:
# read data
data_directory = "./data"
documents = load_pdf(data_directory)
# print(documents)

In [10]:
# create text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [12]:
# split documents
text_chunks = text_splitter.split_documents(documents)